<font size=8>
    <center>FIFA Crawler</center>
</font>
<br>

 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Librairies" data-toc-modified-id="Librairies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Librairies</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Fifa-Players-Crawler" data-toc-modified-id="Fifa-Players-Crawler-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Fifa Players Crawler</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Data-Processing" data-toc-modified-id="Data-Processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Processing</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Change-Lastname-&amp;-Add-Country-Trigramme" data-toc-modified-id="Change-Lastname-&amp;-Add-Country-Trigramme-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Change Lastname &amp; Add Country Trigramme</a></span></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Manage-Duplicates-Lastname" data-toc-modified-id="Manage-Duplicates-Lastname-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Manage Duplicates Lastname</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/Crawler/Team_Crawler_Fifa.ipynb#Pictures-Saver" data-toc-modified-id="Pictures-Saver-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Pictures Saver</a></span></li></ul></div>

# Librairies

In [1]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import pandas as pd
import numpy as np
import requests
import time

# Fifa Players Crawler

In [2]:
def get_team_df(dict_correspondances, countries):
    player_name, player_country, player_position, player_age, player_description, player_link = [], [], [], [], [], []
    browser = webdriver.Firefox()
    browser.get("https://www.fifa.com/worldcup/teams/")
    time.sleep(1)
    
    for country in countries:
        element = browser.find_elements_by_xpath("//div[@class='fi-team-card__flag']")[dict_correspondances[country]]
        browser.execute_script("arguments[0].click();", element)
        time.sleep(1)

        for p in range(23):
            try:
                player = browser.find_elements_by_xpath("//div[@class='fi-p']/a[@class='fi-p--link']")[p]
                new_url = player.get_attribute('href')
                browser.get(new_url)
                time.sleep(1)
                player_name.append(browser.find_element_by_xpath("//div[@class='fi-p__name']").text)
                print(player_name[-1])
                player_country.append(country)
                player_position.append(browser.find_element_by_xpath("//div[@class='fi-p__role']").text)
                player_age.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-number__number']").text)
                try:
                    player_description.append(browser.find_element_by_xpath("//div[@class='fi-p__profile-text']").text)
                except NoSuchElementException:
                    player_description.append("")
                my_img = browser.find_elements_by_tag_name('image')
                for image in my_img:
                    link = image.get_attribute('xlink:href')
                    player_link.append(link)
                browser.back()
                time.sleep(1)
            except IndexError:
                print()

        browser.get("https://www.fifa.com/worldcup/teams/")
        time.sleep(2)
    
    df = pd.DataFrame({"Name":player_name, "Country":player_country, "Position":player_position, "Age":player_age, 
                   "Description":player_description, "Link":player_link})
    df["Firstname"] = df["Name"].apply(lambda x: x.split()[0].capitalize())
    df["Lastname"] = df["Name"].apply(lambda x: "".join(x.split()[1:]).capitalize())
    return df

dict_correspondances = {"Argentina": 0, "Australia": 1, "Belgium": 2, "Brazil": 3,
                        "Colombia": 4, "Costa": 5, "Croatia": 6,
                        "Denmark": 7, "Egypt": 8, "England": 9,
                        "France": 10, "Germany": 11, "Iran": 13, "Japan": 14, "Mexico":16,
                        "Morocco": 17, "Peru": 20, "Poland": 21,
                        "Portugal": 22, "Russia": 23, "Arabia": 24,
                        "Senegal":25, "Serbia": 26, "Spain": 27, 
                        "Sweden": 28, "Switzerland": 29, "Tunisia": 30, 
                        "Uruguay": 31}

df = get_team_df(dict_correspondances, ["Japan"])

EIJI KAWASHIMA
MASAAKI HIGASHIGUCHI
KOSUKE NAKAMURA
NAOMICHI UEDA
GEN SHOJI
YUTO NAGATOMO
WATARU ENDO
HIROKI SAKAI
TOMOAKI MAKINO
GOTOKU SAKAI
MAYA YOSHIDA
KEISUKE HONDA
GAKU SHIBASAKI
GENKI HARAGUCHI
SHINJI KAGAWA
TAKASHI USAMI
TAKASHI INUI
HOTARU YAMAGUCHI
MAKOTO HASEBE
RYOTA OHSHIMA
SHINJI OKAZAKI
YOSHINORI MUTO
YUYA OSAKO


In [6]:
df.to_csv("df_full.csv", index=False)
df.shape

(612, 8)

# Data Processing
## Change Lastname & Add Country Trigramme

In [7]:
df = pd.read_csv("df_full.csv")
print(df.shape)
df.head()

(612, 8)


,Age,Country,Description,Link,Name,Position,Firstname,Lastname
0,35,Japan,The vastly experienced Eiji Kawashima has been...,https://api.fifa.com/api/v1/picture/players/20...,EIJI KAWASHIMA,Goalkeeper,Eiji,Kawashima
1,32,Japan,While Masaaki Higashiguchi remains one of the ...,https://api.fifa.com/api/v1/picture/players/20...,MASAAKI HIGASHIGUCHI,Goalkeeper,Masaaki,Higashiguchi
2,23,Japan,A graduate of the country’s youth ranks who im...,https://api.fifa.com/api/v1/picture/players/20...,KOSUKE NAKAMURA,Goalkeeper,Kosuke,Nakamura
3,23,Japan,Although his appearances for the senior Samura...,https://api.fifa.com/api/v1/picture/players/20...,NAOMICHI UEDA,Defender,Naomichi,Ueda
4,25,Japan,A player who has long been spoken of as a pote...,https://api.fifa.com/api/v1/picture/players/20...,GEN SHOJI,Defender,Gen,Shoji


In [8]:
def get_lastname(text):
    text = text.split()
    if len(text) == 1:
        return text[0].capitalize()
    else:
        return ''.join(text[1:]).capitalize()
    
def get_trigramme(text):
    return dict_trig[text]

dict_trig = {"Argentina": 'ARG', "Australia": 'AUS', "Belgium": 'BEL', "Brazil": 'BRA', "Colombia": 'COL', 
             "Costa": 'CRC', "Croatia": 'CRO', "Denmark": 'DEN', "Egypt": 'EGY', "England": 'ENG',
             "France": 'FRA', "Germany": 'GER', "Iran": 'IRN', "Morocco": 'MAR', "Peru": 'PER', 
             "Poland": 'POL', "Portugal": 'POR', "Russia": 'RUS', "Arabia": 'KSA', "Serbia": 'SER', 
             "Spain": 'SPA', "Sweden": 'SWE', "Switzerland": 'SWI', "Tunisia": 'TUN','United_States':'USA',
             "Senegal":"SEN", "Uruguay":"URU", "Mexico":"MEX", "Japan": "JPN"}

df["Lastname"] = df["Name"].apply(lambda x: get_lastname(x))
df["Trigramme"] = df["Country"].apply(lambda x: get_trigramme(x))

## Manage Duplicates Lastname

In [9]:
print(df["Lastname"].value_counts()[:16])

Silva         7
Fernandes     4
Rodriguez     4
Guzman        3
Sanchez       3
Hawsawi       3
Dimaria       2
Pavon         2
Biglia        2
Kalinic       2
Hernandez     2
Dybala        2
Aquino        2
Tagliafico    2
Miranchuk     2
Jorgensen     2
Name: Lastname, dtype: int64


In [12]:
df[df["Country"]=="Japan"]

,Age,Country,Description,Link,Name,Position,Firstname,Lastname,Trigramme
0,35,Japan,The vastly experienced Eiji Kawashima has been...,https://api.fifa.com/api/v1/picture/players/20...,EIJI KAWASHIMA,Goalkeeper,Eiji,Kawashima,JPN
1,32,Japan,While Masaaki Higashiguchi remains one of the ...,https://api.fifa.com/api/v1/picture/players/20...,MASAAKI HIGASHIGUCHI,Goalkeeper,Masaaki,Higashiguchi,JPN
2,23,Japan,A graduate of the country’s youth ranks who im...,https://api.fifa.com/api/v1/picture/players/20...,KOSUKE NAKAMURA,Goalkeeper,Kosuke,Nakamura,JPN
3,23,Japan,Although his appearances for the senior Samura...,https://api.fifa.com/api/v1/picture/players/20...,NAOMICHI UEDA,Defender,Naomichi,Ueda,JPN
4,25,Japan,A player who has long been spoken of as a pote...,https://api.fifa.com/api/v1/picture/players/20...,GEN SHOJI,Defender,Gen,Shoji,JPN
5,31,Japan,Yuto Nagatomo is now a decade into his Japan n...,https://api.fifa.com/api/v1/picture/players/20...,YUTO NAGATOMO,Defender,Yuto,Nagatomo,JPN
6,25,Japan,Wataru Endo came into the national team reckon...,https://api.fifa.com/api/v1/picture/players/20...,WATARU ENDO,Defender,Wataru,Endo,JPN
7,28,Japan,"After not getting any game-time in Brazil, ful...",https://api.fifa.com/api/v1/picture/players/20...,HIROKI SAKAI,Defender,Hiroki,Sakai,JPN
8,31,Japan,"A veteran of the 2007 FIFA U-20 World Cup, and...",https://api.fifa.com/api/v1/picture/players/20...,TOMOAKI MAKINO,Defender,Tomoaki,Makino,JPN
9,27,Japan,Gotoku Sakai is one of a host of Japan players...,https://api.fifa.com/api/v1/picture/players/20...,GOTOKU SAKAI,Defender,Gotoku,Sakai,JPN


In [13]:
df.loc[df.Name == "BERNARDO SILVA", 'Lastname'] = "Bernardo"
df.loc[df.Name == "ADRIEN SILVA", 'Lastname'] = "Adrien"
df.loc[df.Name == "ANDRE SILVA", 'Lastname'] = "Andre"
df.loc[df.Name == "BRUNO FERNANDES", 'Lastname'] = "Bruno"
df.loc[df.Name == "OMAR HAWSAWI", 'Lastname'] = "Omar"
df.loc[df.Name == "MOTAZ HAWSAWI", 'Lastname'] = "Motaz"
df.loc[df.Name == "ALEXEY MIRANCHUK", 'Lastname'] = "Alexey"
df.loc[df.Name == "NOUREDDINE AMRABAT", 'Lastname'] = "Noureddine"
df.loc[df.Name == "MATHIAS JORGENSEN", 'Lastname'] = "Mathias"
df.loc[df.Name == "JOSE CUADRADO", 'Lastname'] = "Jose"
df.loc[df.Name == "LOVRE KALINIC", 'Lastname'] = "Lovre"
df.loc[df.Name == "DAVINSON SANCHEZ", 'Lastname'] = "Davinson"
df.loc[df.Name == "THORGAN HAZARD", 'Lastname'] = "Thorgan"
df.loc[df.Name == "KHADIM NDIAYE", 'Lastname'] = "Khadim"
df.loc[df.Name == "ALFRED NDIAYE", 'Lastname'] = "Alfred"
df.loc[df.Name == "PAPE ALIOUNE NDIAYE", 'Lastname'] = "Alioune"
df.loc[df.Name == "GASTON SILVA", 'Lastname'] = "Gaston"
df.loc[df.Name == "SERGEJ MILINKOVIC-SAVIC", 'Lastname'] = "Savic"
df.loc[df.Name == "KEVIN DEBRUYNE", 'Lastname'] = "Bruyne"
df.loc[df.Name == "GIOVANI DOS SANTOS", 'Lastname'] = "Giovani"
df.loc[df.Name == "JONATHAN DOS SANTOS", 'Lastname'] = "Jonathan"
df.loc[df.Name == "GOTOKU SAKAI", 'Lastname'] = "Gotoku"

In [14]:
df_worldcup = df.rename(columns={"Trigramme":"Code","Position":"Post"}).copy()
df_worldcup.to_csv("data_worldcup.csv", index=False)

# Pictures Saver

In [3]:
!ls

Team_Crawler_Fifa.ipynb data_worldcup.xlsx      df_full.csv
data_worldcup.csv       data_worldcup_old.csv


In [15]:
df_worldcup = pd.read_csv("data_worldcup.csv")
df_worldcup.head()

,Age,Country,Description,Link,Name,Post,Firstname,Lastname,Code
0,35,Japan,The vastly experienced Eiji Kawashima has been...,https://api.fifa.com/api/v1/picture/players/20...,EIJI KAWASHIMA,Goalkeeper,Eiji,Kawashima,JPN
1,32,Japan,While Masaaki Higashiguchi remains one of the ...,https://api.fifa.com/api/v1/picture/players/20...,MASAAKI HIGASHIGUCHI,Goalkeeper,Masaaki,Higashiguchi,JPN
2,23,Japan,A graduate of the country’s youth ranks who im...,https://api.fifa.com/api/v1/picture/players/20...,KOSUKE NAKAMURA,Goalkeeper,Kosuke,Nakamura,JPN
3,23,Japan,Although his appearances for the senior Samura...,https://api.fifa.com/api/v1/picture/players/20...,NAOMICHI UEDA,Defender,Naomichi,Ueda,JPN
4,25,Japan,A player who has long been spoken of as a pote...,https://api.fifa.com/api/v1/picture/players/20...,GEN SHOJI,Defender,Gen,Shoji,JPN


In [16]:
def image_downloader(name, team, link):
    path = "../DjangoWorldCup/worldcup/static/worldcup/img/players/"
    with open(path + team.capitalize() + '/' + name.capitalize() + '.jpg', 'wb') as handle:
            response = requests.get(link, stream=True)

            if not response.ok:
                print(response)

            for block in response.iter_content(1024):
                if not block:
                    break

                handle.write(block)

for country in ["Japan"]:
    for n, t, l in zip(df_worldcup[df_worldcup["Country"] == country]["Lastname"].tolist(), 
                       df_worldcup[df_worldcup["Country"] == country]["Country"].tolist(), 
                       df_worldcup[df_worldcup["Country"] == country]["Link"].tolist()):
        image_downloader(n, t, l)